In [ ]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version


setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 63.2 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
    
spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
lines = sc.textFile('/content/sonnet.txt')
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)

In [ ]:
counts.collect()

[('hi ', 1), ('how ', 1), ('are', 1), ('hello ', 1), ('you', 1)]

In [ ]:
pairs = lines.map(lambda x: (x.split(" ")[0], x))

In [ ]:
print(pairs.collect())

[('hi', 'hi '), ('hello', 'hello '), ('how', 'how '), ('are', 'are'), ('you', 'you')]


In [ ]:
pairs = lines.map(lambda x: (x.split(" "), x))

In [ ]:
print(pairs.collect())

[(['hi', ''], 'hi '), (['hello', ''], 'hello '), (['how', ''], 'how '), (['are'], 'are'), (['you'], 'you')]


In [ ]:
pairs = lines.map(lambda x: (x.split(" ")[0]))

In [ ]:
print(pairs.collect())

['hi', 'hello', 'how', 'are', 'you']


In [ ]:
counts = pairs.reduceByKey(lambda a, b: a + b)

In [ ]:
counts.collect

<bound method RDD.collect of PythonRDD[14] at collect at <ipython-input-12-d0a4bc5d2775>:1>

In [ ]:
counts

PythonRDD[14] at collect at <ipython-input-12-d0a4bc5d2775>:1

## Sorting Data

In [ ]:
lines.collect()

['hi ', 'hello ', 'how ', 'are', 'you']

In [ ]:
lines.take(5)

['hi ', 'hello ', 'how ', 'are', 'you']

In [ ]:
def Func(lines):
      lines = lines.lower()
      lines = lines.split()
      return lines
rdd1 =lines.map(Func)

In [ ]:
rdd1.take(5)

[['hi'], ['hello'], ['how'], ['are'], ['you']]

In [ ]:
rdd2 = rdd1.groupBy(lambda w: w[0:3])

In [ ]:
rdd2 = lines.groupBy(lambda w: w[0:4])

In [ ]:
rdd3_mapped = rdd2.map(lambda x: (x,1))
rdd3_grouped = rdd3_mapped.groupByKey()

In [ ]:
print(list((j[0], list(j[1])) for j in rdd3_grouped.take(5)))

[(('hell', <pyspark.resultiterable.ResultIterable object at 0x7f4fba4d3450>), [1]), (('you', <pyspark.resultiterable.ResultIterable object at 0x7f4fba4d38d0>), [1]), (('hi ', <pyspark.resultiterable.ResultIterable object at 0x7f4fba0c5790>), [1]), (('how ', <pyspark.resultiterable.ResultIterable object at 0x7f4fba0c5fd0>), [1]), (('are', <pyspark.resultiterable.ResultIterable object at 0x7f4fba098fd0>), [1])]


In [ ]:
rdd3_grouped.collect()

[(('hell', <pyspark.resultiterable.ResultIterable at 0x7f4fba0c5750>),
  <pyspark.resultiterable.ResultIterable at 0x7f4fba0c5e50>),
 (('you', <pyspark.resultiterable.ResultIterable at 0x7f4fba0c5090>),
  <pyspark.resultiterable.ResultIterable at 0x7f4fba098d10>),
 (('hi ', <pyspark.resultiterable.ResultIterable at 0x7f4fba098f10>),
  <pyspark.resultiterable.ResultIterable at 0x7f4fba098f50>),
 (('how ', <pyspark.resultiterable.ResultIterable at 0x7f4fba098ed0>),
  <pyspark.resultiterable.ResultIterable at 0x7f4fba115a90>),
 (('are', <pyspark.resultiterable.ResultIterable at 0x7f4fba112910>),
  <pyspark.resultiterable.ResultIterable at 0x7f4fba1127d0>)]

In [ ]:
rdd3_mapped.reduceByKey(lambda x,y: x+y).map(lambda x:(x[1],x[0])).sortByKey(False).take(10)

[(1, ('hell', <pyspark.resultiterable.ResultIterable at 0x7f4fba0e1610>)),
 (1, ('you', <pyspark.resultiterable.ResultIterable at 0x7f4fba0e1590>)),
 (1, ('hi ', <pyspark.resultiterable.ResultIterable at 0x7f4fba0e1550>)),
 (1, ('how ', <pyspark.resultiterable.ResultIterable at 0x7f4fba0e1290>)),
 (1, ('are', <pyspark.resultiterable.ResultIterable at 0x7f4fba0e18d0>))]

In [ ]:
rdd2.count()

5

In [ ]:
lines.count()

5

In [ ]:
lines.map(lambda k: (k,1)).countByKey().items()

dict_items([('hi ', 1), ('hello ', 1), ('how ', 1), ('are', 1), ('you', 1)])

## InnerJoin 

In [ ]:
valuesA = [('simon',1),('hemal',2),('jeffry',3),('ghladin',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])
 
valuesB = [('shankar',1),('simon',2),('shyam',3),('jeffry',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])
 
TableA.show()
TableB.show()

+-------+---+
|   name| id|
+-------+---+
|  simon|  1|
|  hemal|  2|
| jeffry|  3|
|ghladin|  4|
+-------+---+

+-------+---+
|   name| id|
+-------+---+
|shankar|  1|
|  simon|  2|
|  shyam|  3|
| jeffry|  4|
+-------+---+



In [ ]:
ta = TableA.alias('ta')
tb = TableB.alias('tb')

In [ ]:
inner_join = ta.join(tb, ta.name == tb.name)
inner_join.show()

+------+---+------+---+
|  name| id|  name| id|
+------+---+------+---+
| simon|  1| simon|  2|
|jeffry|  3|jeffry|  4|
+------+---+------+---+



## Left OuterJoin

In [ ]:
left_join = ta.join(tb, ta.name == tb.name,how='left_outer')
left_join.show()

+-------+---+------+----+
|   name| id|  name|  id|
+-------+---+------+----+
|ghladin|  4|  null|null|
|  simon|  1| simon|   2|
|  hemal|  2|  null|null|
| jeffry|  3|jeffry|   4|
+-------+---+------+----+



## Right OuterJoin

In [ ]:
right_join = ta.join(tb, ta.name == tb.name,how='right_outer') 
right_join.show()

+------+----+-------+---+
|  name|  id|   name| id|
+------+----+-------+---+
| simon|   1|  simon|  2|
|  null|null|  shyam|  3|
|jeffry|   3| jeffry|  4|
|  null|null|shankar|  1|
+------+----+-------+---+



In [ ]:
Teams = sc.parallelize(("simon", "hemal", "jeffry", "simon", "ghladin", "jeffry", "join", "simon"))
Teams.map(lambda k: (k,1)).countByKey().items()

dict_items([('simon', 3), ('hemal', 1), ('jeffry', 2), ('ghladin', 1), ('join', 1)])